In [ ]:
# These are the only packages you are allowed import:
import csv
import numpy as np

def my_name():
    return "Rucsanda Juncu"

def parse_network(filename):
    """
    parse_network reads in a file and stores edges bidirectionally. Both prot_list and edges contain no repeated values.
    filename is a string that holds the file name.
    Returns a list of proteins and a set of edges.
    """
    file = open(filename,'r')
    raw_file = csv.reader(file, delimiter="\t")
    proteins = []
    edges = set()

    for i in raw_file:
      edges.add((i[0],i[1]))
      edges.add((i[1],i[0]))

    for index, value in enumerate(edges):
        if any(p == value[0] for p in proteins):
          continue
        else:
          proteins.append(value[0])

    file.close()
    proteins = sorted(proteins)
    return proteins, edges

In [ ]:
def build_laplacian(protein_list, edges):
    """
    build_laplacian takes the list protein_list and set edges from parse_network()
    returns a laplacian matrix
    A=Adjacency matrix containing boolean representation of which proteins are connected
    D=Diagonal matrix containing the sum of the columns of A
    L=Laplacian matrix
    proteins are stored in a dictionary so that each protein is associated with a column/row in a matrix
    """
    size = len(protein_list)
    A = np.zeros((size, size))
    D = np.zeros((size, size))
    prot_dictionary = {}

    for index, prot in enumerate(protein_list):
      prot_dictionary[prot] = index

    for i, pairs in enumerate(edges):
      x = prot_dictionary[pairs[0]]
      y = prot_dictionary[pairs[1]]
      A[x, y] = 1

    sumA = A.sum(axis = 0)

    for d, i in enumerate(D):
      D[d, d] = sumA[d]

    L = D - A
    return L



In [ ]:
def connected_components(laplacian):
    """
    connected_components takes the numpy laplacian matrix from build_laplacian
    returns an int containing the number of zero eigenvalues in the matrix
    e-8 is used as a cut off for zero eigenvalues since no lambda is truly equal to 0
    eigvalsh is used since L is a hermitian matrix (L=L^T)
    zero eigenvalues = number of connected components
    """
    eigs = np.linalg.eigvalsh(laplacian)
    zero_eigs = 0
    for i in eigs:
      if i <= 0.000000001:
        zero_eigs = zero_eigs+1
    return zero_eigs


In [ ]:
def get_neighborhood(protein, edges):
    """
    get_neighborhood takes a character that represents a protein in the network, and a set of edges
    returns a set of proteins connected to the passed protein
    """
    neighbors = set()
    for e in edges:
      if protein == e[0]:
        neighbors.add(e[0])
        neighbors.add(e[1])

    neighbors = sorted(neighbors)
    return set(neighbors)

In [ ]:
def detect_cluster(protein, edges):
    """
    detect_cluster takes a char that represents a protein and a set of edges
    returns a float clustering_coefficient and the number of proteins in a neighborhood as int nodes
    clusters computed using the formula 2|E|/(k(k-1))
    E = number of edges in a neighborhood
    k = number of vertices
    clustering_coefficient of 1 indicates a completely connected cluster
    """
    neighbors = get_neighborhood(protein, edges)
    nodes = len(neighbors)
    temp_edge = list()

    for node in neighbors:
      for edge in edges:
        if node == edge[0] and edge[1] in neighbors:
          temp_edge.append(edge)

    clustering_coefficient = len(temp_edge)/(nodes*(nodes-1))
    return clustering_coefficient, nodes

In [ ]:
def get_all_clusters(protein_list, edges):
  """
  get_all_clusters takes a list of all proteins and a set of all edges
  returns a list clusters that contains tuples with the protein, number of neighbors, and the clustering coefficient
  selects clusters with coefficients >= .90 and sorts them by how many neighbors are in the cluster
  """
  clusters = list()

  for prot in protein_list:
    neighbors = get_neighborhood(prot, edges)
    if len(neighbors) > 3:
      clustering_coefficient, nodes = detect_cluster(prot, edges)
      if clustering_coefficient >= 0.90:
        clusters.append(tuple((prot, len(neighbors), clustering_coefficient)))

  clusters = sorted(clusters, key = lambda x: x[1])
  return clusters

In [ ]:
if __name__ == "__main__":
    prot_list, edges = parse_network("example_network.tsv")
    # laplace_matrix = build_laplacian(prot_list, edges)
    # neighbors = get_neighborhood('D', edges)
    # print(detect_cluster('D', edges))
    # get_all_clusters(prot_list, edges)
    # print(neighbors)
    print(edges)

{('D', 'F'), ('B', 'A'), ('C', 'E'), ('E', 'C'), ('C', 'B'), ('D', 'B'), ('B', 'C'), ('D', 'C'), ('C', 'D'), ('A', 'D'), ('E', 'B'), ('C', 'A'), ('A', 'C'), ('B', 'D'), ('A', 'B'), ('D', 'A'), ('F', 'D'), ('B', 'E')}
